In [6]:
import pandas as pd
import psycopg2 as pg
import pandas.io.sql as psql
import datetime as DT
from datetime import date, timedelta
import pytz
import requests

In [3]:
#!pip3 install psycopg2

In [13]:
#Gera a data e hora de São Paulo Brasil
tz_BR = pytz.timezone('America/Sao_Paulo')
#Gera a data atual em formato Timestamp
start = DT.datetime.strftime(DT.datetime.now(tz_BR),'%Y-%m-%d %H:%M:%S')
print("Processo iniciado em: "+start)

#PostgreSQL:
#•	Host: 34.173.103.16
#•	User: junior
#•	Password: |?7LXmg+FWL&,2(
#•	Port: 5432
#•	Database: postgres
#•	Tabela: public.venda

#Inicia a configuração para conexão com as tabelas do postgressql
connection = pg.connect("host=34.173.103.16 dbname=postgres user=junior password=|?7LXmg+FWL&,2(")

#Executa Query para retornar o resultado e armazenar um Dataframe Pandas
#Query com solução para os dados solicitados
QUERY = """SELECT * FROM public.venda"""
print(QUERY)
df_postgressql_pd = psql.read_sql_query(QUERY, connection)
end_consulta = DT.datetime.strftime(DT.datetime.now(tz_BR),'%Y-%m-%d %H:%M:%S')
print("Consulta realizada com Sucesso: "+end_consulta)
df_postgressql_pd

Processo iniciado em: 2022-10-03 12:37:36
SELECT * FROM public.venda


C:\Users\caior_op46gft\anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


Consulta realizada com Sucesso: 2022-10-03 12:37:39


,id_venda,id_funcionario,id_categoria,data_venda,venda
0,1,1,1,2017-10-01,21636
1,2,1,4,2018-05-12,3312
2,3,1,3,2019-02-01,11778
3,4,1,4,2019-03-11,2554
4,5,1,3,2018-09-07,4425
...,...,...,...,...,...
1012,1013,9,1,2017-02-08,10689
1013,1014,9,3,2018-03-07,15056
1014,1015,9,6,2019-06-03,3101
1015,1016,9,4,2020-02-03,204


In [19]:
#comando para chamar a api
df_funcionarios = pd.DataFrame()

for number in range(1, 10):
    url = f'https://us-central1-bix-tecnologia-prd.cloudfunctions.net/api_challenge_junior?id={number}'
    resp = requests.get(url)
    funcionario = resp.text
    df_funcionario_api = pd.DataFrame({'funcionario': [funcionario],
                                       'id_funcionario': [number]})
    df_funcionarios = pd.concat([df_funcionarios,df_funcionario_api])

In [20]:
df_funcionarios

,funcionario,id_funcionario
0,Rob Carsson,1
0,Eli Preston,2
0,Tom Lindwall,3
0,Leif Shine,4
0,Ingrid Hendrix,5
0,Lennart Skoglund,6
0,Rock Rollman,7
0,Helen Brolin,8
0,Joan Callins,9


In [26]:
#Comando para leitura do parquet em pandas
path_parquet_file = "C:\\Users\\caior_op46gft\\Desktop\\arquivo_parquet\\"
df_parquet_pd = pd.read_parquet(path_parquet_file+'categoria.parquet')
df_parquet_pd

,id,nome_categoria
0,1,Babywear
1,2,Womens wear
2,3,Womens Footwear
3,4,Sportwear
4,5,Mens Clothes
5,6,Bath Clothes
6,7,Mens Footwear
7,8,Childrens wear


In [49]:
df_tabela_final= df_postgressql_pd.merge(df_funcionarios)\
                                  .merge(df_parquet_pd, left_on='id_categoria', right_on='id',\
                                         how='left').drop('id', axis=1)

In [50]:
df_tabela_final

,id_venda,id_funcionario,id_categoria,data_venda,venda,funcionario,nome_categoria
0,1,1,1,2017-10-01,21636,Rob Carsson,Babywear
1,2,1,4,2018-05-12,3312,Rob Carsson,Sportwear
2,3,1,3,2019-02-01,11778,Rob Carsson,Womens Footwear
3,4,1,4,2019-03-11,2554,Rob Carsson,Sportwear
4,5,1,3,2018-09-07,4425,Rob Carsson,Womens Footwear
...,...,...,...,...,...,...,...
1012,1013,9,1,2017-02-08,10689,Joan Callins,Babywear
1013,1014,9,3,2018-03-07,15056,Joan Callins,Womens Footwear
1014,1015,9,6,2019-06-03,3101,Joan Callins,Bath Clothes
1015,1016,9,4,2020-02-03,204,Joan Callins,Sportwear
